<a href="https://colab.research.google.com/github/safiqurrahmang7/Project-Fake_news_classifier/blob/main/notebooks/Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
df = pd.read_csv('/content/fake_news_processed.csv')

In [2]:
df.head()

,text,label,processed_text
0,no comment is expected from barack obama membe...,1,no comment is expected from barack obama membe...
1,did they post their votes for hillary already,1,did they post their vote for hillary already
2,now most of the demonstrators gathered last n...,1,now most of the demonstrator gathered last nig...
3,a dozen politically active pastors came here f...,0,a dozen politically active pastor came here fo...
4,the rs28 sarmat missile dubbed satan 2 will re...,1,the rs28 sarmat missile dubbed satan 2 will re...


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
df['text'] = df['text'].astype(str)

In [5]:
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['text'])
vocab_size = len(tokenizer.word_index)

In [6]:
vocab_size = min(10000, len(tokenizer.word_index) + 1)

In [7]:
sequences = tokenizer.texts_to_sequences(df['text'])
sequence_lengths = [len(seq) for seq in sequences]

In [8]:
max_length = max(sequence_lengths)

In [9]:
padded_text = pad_sequences(sequences,maxlen = 100, padding='post', truncating='post')

print(padded_text.shape)

(62718, 100)


In [10]:
from sklearn.model_selection import train_test_split


# Replace `labels` with your actual labels for the dataset
labels = df['label'].values  # Use the actual column for labels

# Split the data
X_train, X_test, y_train, y_test = train_test_split(padded_text, labels, test_size=0.2, random_state=42)


In [11]:
print('the shape of X_train:',X_train.shape)
print('the shape of y_train:',y_train.shape)


the shape of X_train: (50174, 100)
the shape of y_train: (50174,)


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, SpatialDropout1D
from tensorflow.keras.optimizers import Adam

model = Sequential()

# Embedding Layer
model.add(Embedding(input_dim=10000, output_dim=100, input_length=100))

# Spatial Dropout (to prevent overfitting)
model.add(SpatialDropout1D(0.2))

# LSTM Layer
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))

# Dense Layer (output layer)
model.add(Dense(1, activation='sigmoid'))  # Binary classification (sigmoid activation)

# Step 6: Compiling the Model
model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])


In [ ]:
# Step 7: Training the Model
history = model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test), verbose=2)


Epoch 1/5


In [ ]:
# Step 8: Evaluating the Model
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Validation Accuracy: {accuracy:.4f}')

In [ ]:
# Save the model if needed
model.save('fake_news_classifier_lstm.h5')